In [1]:
sc

In [2]:
!rm -rf metastore_db/*.lck
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

## Decision Trees

### Classification

In [4]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

!wget https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_libsvm_data.txt

--2016-10-01 16:23:35--  https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_libsvm_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.12.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.12.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104736 (102K) [text/plain]
Saving to: ‘sample_libsvm_data.txt.3’

100%[======================================>] 104.736     --.-K/s   in 0,09s   

2016-10-01 16:23:36 (1,13 MB/s) - ‘sample_libsvm_data.txt.3’ saved [104736/104736]



In [5]:
data = MLUtils.loadLibSVMFile(sc, 'sample_libsvm_data.txt')

trainingData, testData = data.randomSplit([0.7, 0.3])

print trainingData.take(1)

labels = testData.map(lambda x: x.label)
features = testData.map(lambda x: x.features)

[LabeledPoint(1.0, (692,[152,153,154,180,181,182,183,208,209,210,211,236,237,238,239,264,265,266,267,292,293,294,295,320,321,322,323,349,350,351,377,378,379,405,406,407,433,434,435,461,462,463,489,490,491,492,517,518,519,520,546,547,548,574,575,576,602,603,604,630,631,632,658,659,660,686,687,688],[5.0,63.0,197.0,20.0,254.0,230.0,24.0,20.0,254.0,254.0,48.0,20.0,254.0,255.0,48.0,20.0,254.0,254.0,57.0,20.0,254.0,254.0,108.0,16.0,239.0,254.0,143.0,178.0,254.0,143.0,178.0,254.0,143.0,178.0,254.0,162.0,178.0,254.0,240.0,113.0,254.0,240.0,83.0,254.0,245.0,31.0,79.0,254.0,246.0,38.0,214.0,254.0,150.0,144.0,241.0,8.0,144.0,240.0,2.0,144.0,254.0,82.0,230.0,247.0,40.0,168.0,209.0,31.0]))]


In [12]:
model = DecisionTree.trainClassifier(trainingData, 
                                     numClasses=2, 
                                     categoricalFeaturesInfo={},
                                     impurity='gini',
                                     maxDepth=5,
                                     maxBins=32)

print(model.toDebugString())

DecisionTreeModel classifier of depth 1 with 3 nodes
  If (feature 434 <= 0.0)
   Predict: 0.0
  Else (feature 434 > 0.0)
   Predict: 1.0



In [13]:
predictions = model.predict(features)

labelsAndPredictions = labels.zip(predictions)

print labelsAndPredictions.collect()

[(1.0, 1.0), (1.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 1.0), (0.0, 0.0), (1.0, 1.0), (0.0, 0.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 0.0), (0.0, 0.0), (1.0, 1.0), (0.0, 0.0), (1.0, 1.0), (1.0, 1.0), (0.0, 0.0), (0.0, 0.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 1.0)]


In [14]:
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())

print('Test Error = ' + str(testErr))

Test Error = 0.0344827586207


In [21]:
!rm -rf myDecisionTreeClassificationModel.parquet
model.save(sc, "myDecisionTreeClassificationModel.parquet")
sameModel = DecisionTreeModel.load(sc, "myDecisionTreeClassificationModel.parquet")

In [28]:
!ls -l myDecisionTreeClassificationModel.parquet/data/

total 32
-rw------- 1 dvgodoy dvgodoy 1468 Set 26 10:01 part-r-00000-a544d8f1-bfcc-4b89-8719-6ba8cb8dc152.snappy.parquet
-rw------- 1 dvgodoy dvgodoy 1468 Set 26 10:01 part-r-00001-a544d8f1-bfcc-4b89-8719-6ba8cb8dc152.snappy.parquet
-rw------- 1 dvgodoy dvgodoy 2807 Set 26 10:01 part-r-00002-a544d8f1-bfcc-4b89-8719-6ba8cb8dc152.snappy.parquet
-rw------- 1 dvgodoy dvgodoy 1468 Set 26 10:01 part-r-00003-a544d8f1-bfcc-4b89-8719-6ba8cb8dc152.snappy.parquet
-rw------- 1 dvgodoy dvgodoy 1468 Set 26 10:01 part-r-00004-a544d8f1-bfcc-4b89-8719-6ba8cb8dc152.snappy.parquet
-rw------- 1 dvgodoy dvgodoy 2599 Set 26 10:01 part-r-00005-a544d8f1-bfcc-4b89-8719-6ba8cb8dc152.snappy.parquet
-rw------- 1 dvgodoy dvgodoy 1468 Set 26 10:01 part-r-00006-a544d8f1-bfcc-4b89-8719-6ba8cb8dc152.snappy.parquet
-rw------- 1 dvgodoy dvgodoy 2599 Set 26 10:01 part-r-00007-a544d8f1-bfcc-4b89-8719-6ba8cb8dc152.snappy.parquet
-rw------- 1 dvgodoy dvgodoy    0 Set 26 10:01 _SUCCESS


In [29]:
!cat myDecisionTreeClassificationModel.parquet/metadata/part-00000

{"class":"org.apache.spark.mllib.tree.DecisionTreeModel","version":"1.0","algo":"Classification","numNodes":3}


### Regression

In [31]:
model = DecisionTree.trainRegressor(trainingData,
                                    categoricalFeaturesInfo={},
                                    impurity='variance',
                                    maxDepth=5, 
                                    maxBins=32)

print(model.toDebugString())

DecisionTreeModel regressor of depth 2 with 5 nodes
  If (feature 434 <= 0.0)
   If (feature 99 <= 0.0)
    Predict: 0.0
   Else (feature 99 > 0.0)
    Predict: 1.0
  Else (feature 434 > 0.0)
   Predict: 1.0



In [32]:
predictions = model.predict(features)

labelsAndPredictions = labels.zip(predictions)

testMSE = labelsAndPredictions.map(lambda (v, p): (v - p) * (v - p)).sum() / float(testData.count())

print('Test Mean Squared Error = ' + str(testMSE))

Test Mean Squared Error = 0.0333333333333
